<a href="https://colab.research.google.com/github/RomaA2000/RecommendationSystems/blob/main/hw1/HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Матричные факторизации

В данной работе вам предстоит познакомиться с практической стороной матричных разложений.
Работа поделена на 4 задания:
1. Вам необходимо реализовать SVD разложения используя SGD на explicit данных
2. Вам необходимо реализовать матричное разложения используя ALS на implicit данных
3. Вам необходимо реализовать матричное разложения используя BPR(pair-wise loss) на implicit данных
4. Вам необходимо реализовать матричное разложения используя WARP(list-wise loss) на implicit данных


In [1]:
from google.colab import drive
drive.mount('/content/drive/')
 
#Выбрать нужный путь до данных


Mounted at /content/drive/


In [2]:
!pip install implicit
!pip install lightfm

     |████████████████████████████████| 1.1MB 15.6MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp37-cp37m-linux_x86_64.whl size=3406364 sha256=ee877e88f533fdbc884d68fe337d1b5d2363d9830ec51d30758a64284390f4d1
  Stored in directory: /root/.cache/pip/wheels/bf/d4/ec/fd4f622fcbefb7521f149905295b2c26adecb23af38aa28217
Successfully built implicit
     |████████████████████████████████| 317kB 13.0MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=706129 sha256=39e9deabe7e6a9080133d15bc125857dbdb3ca938424d3dca4ff2659d75474dc
  Stored in directory: /root/.cache/pip/wheels/c6/64/d4/673c7277f71ac4c5ad4835b94708c01b653ef2d3aa78ef20aa
Successfully built lightfm


In [3]:
import implicit
import pandas as pd
import numpy as np
import scipy.sparse as sp
import os
import torch

from lightfm.datasets import fetch_movielens

В данной работе мы будем работать с explicit датасетом movieLens, в котором представленны пары user_id movie_id и rating выставленный пользователем фильму

Скачать датасет можно по ссылке https://grouplens.org/datasets/movielens/1m/

In [ ]:
data_dir = 'RecSysHSE/ml-1m'

In [4]:
ratings = pd.read_csv(os.path.join(data_dir, 'ratings.dat'), delimiter='::', header=None, 
        names=['user_id', 'movie_id', 'rating', 'timestamp'], 
        usecols=['user_id', 'movie_id', 'rating'], engine='python')

In [5]:
movie_info = pd.read_csv(os.path.join(data_dir, 'movies.dat'), delimiter='::', header=None, 
        names=['movie_id', 'name', 'category'], engine='python')

Explicit данные

Для того, чтобы преобразовать текущий датасет в Implicit, давайте считать что позитивная оценка это оценка >=4

In [6]:
implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]

In [7]:
implicit_ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
3,1,3408,4
4,1,2355,5
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4
10,1,595,5
11,1,938,4
12,1,2398,4


Удобнее работать с sparse матричками, давайте преобразуем DataFrame в CSR матрицы

In [8]:
users = implicit_ratings["user_id"]
movies = implicit_ratings["movie_id"]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

В качестве примера воспользуемся ALS разложением из библиотеки implicit

Зададим размерность латентного пространства равным 64, это же определяет размер user/item эмбедингов

In [9]:
model = implicit.als.AlternatingLeastSquares(factors=64, iterations=100, calculate_training_loss=True)

В качестве loss здесь всеми любимый RMSE

In [10]:
model.fit(user_item_t_csr)

Построим похожие фильмы по 1 movie_id = Истории игрушек

In [11]:
movie_info.head(5)

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [12]:
get_similars = lambda item_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                        for x in model.similar_items(item_id)]

Как мы видим, симилары действительно оказались симиларами.

Качество симиларов часто является хорошим способом проверить качество алгоритмов.

P.S. Если хочется поглубже разобраться в том как разные алгоритмы формируют разные латентные пространства, рекомендую загружать полученные вектора в tensorBoard и смотреть на сформированное пространство

In [13]:
get_similars(1, model)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 "2286    Bug's Life, A (1998)",
 '33    Babe (1995)',
 '2315    Babe: Pig in the City (1998)',
 '584    Aladdin (1992)',
 '1526    Hercules (1997)',
 '3817    Went to Coney Island on a Mission From God... ...',
 '2252    Pleasantville (1998)',
 '2692    Iron Giant, The (1999)']

Давайте теперь построим рекомендации для юзеров

Как мы видим юзеру нравится фантастика, значит и в рекомендациях ожидаем увидеть фантастику

In [14]:
get_user_history = lambda user_id, implicit_ratings : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
                                            for x in implicit_ratings[implicit_ratings["user_id"] == user_id]["movie_id"]]

In [15]:
get_user_history(4, implicit_ratings)

['3399    Hustler, The (1961)',
 '2882    Fistful of Dollars, A (1964)',
 '1196    Alien (1979)',
 '1023    Die Hard (1988)',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '1959    Saving Private Ryan (1998)',
 '476    Jurassic Park (1993)',
 '1180    Raiders of the Lost Ark (1981)',
 '1885    Rocky (1976)',
 '1081    E.T. the Extra-Terrestrial (1982)',
 '3349    Thelma & Louise (1991)',
 '3633    Mad Max (1979)',
 '2297    King Kong (1933)',
 '1366    Jaws (1975)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '2623    Run Lola Run (Lola rennt) (1998)',
 '2878    Goldfinger (1964)',
 '1220    Terminator, The (1984)']

Получилось! 

Мы действительно порекомендовали пользователю фантастику и боевики, более того встречаются продолжения тех фильмов, которые он высоко оценил

In [16]:
get_recommendations = lambda user_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                               for x in model.recommend(user_id, user_item_csr)]

In [17]:
get_recommendations(4, model)

['585    Terminator 2: Judgment Day (1991)',
 '1271    Indiana Jones and the Last Crusade (1989)',
 '1284    Butch Cassidy and the Sundance Kid (1969)',
 '1182    Aliens (1986)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '2502    Matrix, The (1999)',
 '1884    French Connection, The (1971)',
 '1892    Rain Man (1988)',
 '957    African Queen, The (1951)',
 '3458    Predator (1987)']

Теперь ваша очередь реализовать самые популярные алгоритмы матричных разложений

Что будет оцениваться:
1. Корректность алгоритма
2. Качество получившихся симиларов
3. Качество итоговых рекомендаций для юзера

In [18]:
from sklearn.neighbors import NearestNeighbors

In [19]:
user_number = np.max(ratings['user_id'])
item_number = np.max(ratings['movie_id'])

In [20]:
# Делит фильмы на просмотренные и не просмотренные для конкретного юзера
def get_un_watched(user_id):
  watched = [i for i in ratings[ratings["user_id"] == user_id]["movie_id"]]
  unwatched = [i for i in movie_info['movie_id'].unique() if i not in watched]
  return unwatched, watched

In [ ]:
# Путь до папки с весами
dir_name = '.'

### Задание 1. Не использую готовые решения, реализовать SVD разложение используя SGD на explicit данных

In [21]:
class SGD:
  def __init__(self, user_number, item_number, lr=0.01, hidden_size=64, l2=0.01):
    self.user_number = user_number
    self.item_number = item_number
    
    self.lr = lr
    self.hidden_size = hidden_size
    self.l2 = l2
    max_value = 1 / np.sqrt(self.hidden_size)
    self.user = np.zeros((user_number, 1))
    self.W = np.random.uniform(0, max_value, (user_number, self.hidden_size))
    self.H = np.random.uniform(0, max_value, (item_number, self.hidden_size)).T
    self.item = np.zeros((item_number, 1)).T

  def fit(self, data, epoch):
    mean = data['rating'].mean()
    for epoch_num in range(epoch):
      random_idx = np.random.randint(len(data))
      data_at_idx = data.iloc[random_idx]
      i, j = data_at_idx['user_id'], data_at_idx['movie_id']
      i -= 1
      j -= 1
      rating_diff = mean - data_at_idx['rating']
      u_vec = self.W[i]
      i_vec = self.H[:, j]
      result = np.matmul(u_vec, i_vec)
      now_user = self.user[i]
      now_item = self.item[:, j]
      error = result + now_user + now_item + rating_diff
      error *= 2
      if epoch_num % (epoch / 10) == 0:
        print(self.rmse(data))
      self.W[i] *= (1 - self.lr * self.l2)
      self.H[:, j] *= (1 - self.lr * self.l2)
      self.user[i] *= (1 - self.lr * self.l2)
      self.item[:, j] *= (1 - self.lr * self.l2)
      self.W[i] -= self.lr * error * i_vec.T
      self.H[:, j] -= self.lr * error * u_vec.T
      self.user[i] -= self.lr * error
      self.item[:, j] -= self.lr * error

  def rmse(self, data):
    result_predicted = np.matmul(self.W, self.H)
    result_predicted += self.item + self.user 
    result_predicted += data['rating'].mean()
    users = data['user_id'] - 1
    items = data['movie_id'] - 1
    raiting_true = data['rating']
    return np.linalg.norm(result_predicted[users, items] - raiting_true) / len(data)

  def save_weights(self, path):
    np.save(os.path.join(path, "W"), self.W)
    np.save(os.path.join(path, "H"), self.H)

  def save_biases(self, path):
    np.save(os.path.join(path, "U"), self.user)
    np.save(os.path.join(path, "I"), self.item)

  def save_model(self, path):
    try: 
      os.mkdir(path)
    except OSError as error: 
      pass
    self.save_weights(path)
    self.save_biases(path)
  
  def load_weights(self, path):
    self.W = np.load(os.path.join(path, "W.npy"))
    self.H = np.load(os.path.join(path, "H.npy"))

  def load_biases(self, path):
    self.user = np.load(os.path.join(path, "U.npy"))
    self.item = np.load(os.path.join(path, "I.npy"))
  
  def load_model(self, path):
    self.load_weights(path)
    self.load_biases(path)

  def get_info(self, info, size, preds_list):
    return [info[info["movie_id"] == i[0]]["name"].to_string() for i in preds_list if i[0] in set(movie_info["movie_id"])][:size]

  def recommend(self, user_id, info, size=10):
    unwatched, _ = get_un_watched(user_id)
    user_info = np.matmul(self.W, self.H)[user_id - 1]
    user_info += self.item.squeeze(0)
    unwatched.sort(key = lambda i: user_info[i - 1], reverse = True)
    return self.get_info(info, size, zip(unwatched[:size], user_info[unwatched[:size]]))

  def similar_items(self, item, info, size=10):
    model = NearestNeighbors(n_neighbors=item_number, metric='cosine')
    vector_item = self.H.T[item - 1].reshape(1, -1)
    dist, idx = model.fit(self.H.T).kneighbors(vector_item)
    return self.get_info(info, size, zip(idx[0] + 1, dist[0]))

In [22]:
sgd = SGD(user_number, item_number)

In [23]:
#sgd.fit(ratings, 3000000)
sgd.load_model(os.path.join(dir_name, "weights", "SGD"))

In [27]:
sgd.similar_items(1, movie_info)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 '817    Adventures of Pinocchio, The (1996)',
 '1015    Mary Poppins (1964)',
 '584    Aladdin (1992)',
 '1988    Incredible Journey, The (1963)',
 '2027    Sleeping Beauty (1959)',
 '591    Beauty and the Beast (1991)',
 '2012    Little Mermaid, The (1989)',
 "2286    Bug's Life, A (1998)"]

In [26]:
sgd.recommend(4, movie_info)

["523    Schindler's List (1993)",
 '847    Godfather, The (1972)',
 '1950    Seven Samurai (The Magnificent Seven) (Shichin...',
 '2836    Sanjuro (1962)',
 '315    Shawshank Redemption, The (1994)',
 '896    North by Northwest (1959)',
 '589    Silence of the Lambs, The (1991)',
 '892    Rear Window (1954)',
 '1195    GoodFellas (1990)',
 '1575    L.A. Confidential (1997)']

### Задание 2. Не использую готовые решения, реализовать матричное разложение используя ALS на implicit данных

In [28]:
class ALS(SGD):
  def __init__(self, user_number, item_number, lr=40, hidden_size=64, l1=0.01):
    self.l1 = l1
    super().__init__(user_number, item_number, lr, hidden_size)

  def fit(self, data, epoch):
    lambda_I = np.identity(self.hidden_size) * self.l1
    users = data["user_id"] - 1
    movies = data["movie_id"] - 1
    p = np.array(sp.coo_matrix((np.ones_like(users), (users, movies))).toarray()).astype(float)
    C = p * self.lr + 1
    for epoch_num in range(epoch):
      if epoch_num % 2 == 0:
        for i in range(self.user_number):
          inversed = np.linalg.inv(np.matmul(self.H * C[i], self.H.T) + lambda_I)
          self.W[i] = np.matmul(np.matmul(inversed, self.H * C[i]), p[i])
      else:
        for i in range(self.item_number):
          inversed = np.linalg.inv(np.matmul(self.W.T * C[:, i], self.W) + lambda_I)
          self.H[:, i] = np.matmul(np.matmul(inversed, self.W.T * C[:, i]), p[:, i])
      if epoch_num % (epoch / 10) == 0:
        print(self.rmse(data))

  def load_model(self, path):
    self.load_weights(path)

  def save_model(self, path):
    try: 
      os.mkdir(path)
    except OSError as error: 
      pass
    self.save_weights(path)

  def rmse(self, data):
    result_predicted = np.matmul(self.W, self.H)
    users = data['user_id'] - 1
    items = data['movie_id'] - 1
    raiting_true = data['rating']
    return np.linalg.norm(result_predicted[users, items] - raiting_true) / len(data)


In [29]:
als = ALS(user_number, item_number)

In [30]:
#als.fit(implicit_ratings, 10)
als.load_model(os.path.join(dir_name, "weights", "ALS"))

In [31]:
als.similar_items(1, movie_info)

['0    Toy Story (1995)',
 '1245    Groundhog Day (1993)',
 '3045    Toy Story 2 (1999)',
 '2327    Shakespeare in Love (1998)',
 '33    Babe (1995)',
 '315    Shawshank Redemption, The (1994)',
 '1959    Saving Private Ryan (1998)',
 '1250    Back to the Future (1985)',
 '2789    American Beauty (1999)',
 '2693    Sixth Sense, The (1999)']

In [32]:
als.recommend(4, movie_info)

['585    Terminator 2: Judgment Day (1991)',
 '2502    Matrix, The (1999)',
 '1182    Aliens (1986)',
 '847    Godfather, The (1972)',
 '1271    Indiana Jones and the Last Crusade (1989)',
 '108    Braveheart (1995)',
 '453    Fugitive, The (1993)',
 '1203    Godfather: Part II, The (1974)',
 '1284    Butch Cassidy and the Sundance Kid (1969)',
 '1179    Princess Bride, The (1987)']

### Задание 3. Не использую готовые решения, реализовать матричное разложение BPR на implicit данных

In [33]:
def sigmoid(x, vector):
  e = np.exp(-x)
  return e / (1 + e) * vector

class BPR(ALS):
  def __init__(self, user_number, item_number, lr=0.01, hidden_size=128, sampling_size=20, l2 = 0.01):
    super().__init__(user_number, item_number, lr, hidden_size, l2)
    self.sampling_size = sampling_size
    
  def fit(self, data, epoch_number):
    not_used_items = {}
    used_items = {}
    unique_mapped_ids = data["user_id"].unique() - 1
    for i in range(user_number):
      not_used_items[i], used_items[i] = get_un_watched(i + 1)
    for epoch in range(epoch_number):
      epoch_loss = 0
      for i in unique_mapped_ids:
        for j in used_items[i]:
          sampling_negative = np.random.choice(not_used_items[i], self.sampling_size)
          j -= 1
          sampling_negative -= 1
          delta = np.matmul(self.W[i], self.H[:, j]) - np.matmul(self.W[i], self.H[:, sampling_negative])
          self.W[i] *= (1 - self.lr * self.l2)
          self.H[:, j] *= (1 - self.lr * self.l2)
          self.H[:, sampling_negative] *= (1 - self.lr * self.l2)
          self.W[i] += self.lr * sigmoid(delta, self.H[:, j].reshape(-1,1) - self.H[:, sampling_negative]).sum(axis=1)
          self.H[:, j] += self.lr * sigmoid(delta, self.W[i].reshape(-1,1)).sum(axis=1)
          self.H[:, sampling_negative] -= self.lr * sigmoid(delta, self.W[i].reshape(-1,1))
          size = len(sampling_negative)
          epoch_loss += np.array(np.log(1 + np.exp(-delta)) / size).sum()
      print("epoch loss: ", epoch_loss / len(unique_mapped_ids))


In [34]:
bpr = BPR(user_number, item_number)

In [35]:
#bpr.fit(implicit_ratings, 1) #7
bpr.load_model(os.path.join(dir_name, "weights", "BPR"))

In [36]:
bpr.similar_items(1, movie_info)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 "2286    Bug's Life, A (1998)",
 '33    Babe (1995)',
 '1245    Groundhog Day (1993)',
 '584    Aladdin (1992)',
 "1854    There's Something About Mary (1998)",
 '2327    Shakespeare in Love (1998)',
 '547    Nightmare Before Christmas, The (1993)',
 '315    Shawshank Redemption, The (1994)']

In [37]:
bpr.recommend(4, movie_info)

['847    Godfather, The (1972)',
 '585    Terminator 2: Judgment Day (1991)',
 '1182    Aliens (1986)',
 '2502    Matrix, The (1999)',
 '1203    Godfather: Part II, The (1974)',
 '604    Fargo (1996)',
 '589    Silence of the Lambs, The (1991)',
 '1271    Indiana Jones and the Last Crusade (1989)',
 '108    Braveheart (1995)',
 '2693    Sixth Sense, The (1999)']

### Задание 4. Не использую готовые решения, реализовать матричное разложение WARP на implicit данных

In [38]:
class WARP(ALS):
  def __init__(self, user_number, item_number, lr=0.01, hidden_size=128, sampling_size=5, l2 = 0.1):
    super().__init__(user_number, item_number, lr, hidden_size, l2)
    self.sampling_size = sampling_size
    
  def fit(self, data, epoch_number):
    not_used_items = {}
    used_items = {}
    unique_mapped_ids = data["user_id"].unique() - 1
    for i in range(user_number):
      not_used_items[i], used_items[i] = get_un_watched(i + 1)
    for epoch in range(epoch_number):
      epoch_loss = 0
      for i in unique_mapped_ids:
        for j in used_items[i]:
          j -= 1
          sampling_negative = np.random.choice(not_used_items[i], self.sampling_size)
          sampling_negative -= 1
          mul_biased = np.matmul(self.W[i], self.H[:, j])
          for q, k in enumerate(sampling_negative):
            delta = mul_biased - np.matmul(self.W[i], self.H[:, k])
            if 1 > delta:
              prod = np.log(self.sampling_size / (q + 1))
              self.W[i] *= (1 - self.lr * self.l2)
              self.H[:, j] *= (1 - self.lr * self.l2)
              self.H[:, k] *= (1 - self.lr * self.l2)
              self.W[i] -= self.lr * prod * (self.H[:, k] - self.H[:, j])
              self.H[:, j] += self.lr * prod * self.W[i]
              self.H[:, k] -= self.lr * prod * self.W[i]
              epoch_loss += prod
      print("epoch loss: ", epoch_loss / len(unique_mapped_ids))


In [39]:
warp = WARP(user_number, item_number)

In [294]:
#warp.fit(implicit_ratings, 40)
warp.load_model(os.path.join(dir_name, "weights", "WARP"))

epoch loss:  40.15938407230962
epoch loss:  38.93994446972263
epoch loss:  37.734790724545164
epoch loss:  36.572683114687024
epoch loss:  35.760396619326805
epoch loss:  34.858443894736766
epoch loss:  34.19657930123258
epoch loss:  33.53754626255669
epoch loss:  32.70561362178007
epoch loss:  32.191549023724015
epoch loss:  31.61486355086118
epoch loss:  31.13515205248801
epoch loss:  30.650099213899455
epoch loss:  30.190501330486267
epoch loss:  29.80177523582929
epoch loss:  29.39441797214002
epoch loss:  28.970437325732625
epoch loss:  28.722402534925685
epoch loss:  28.418993868463712
epoch loss:  27.936011561275276


In [41]:
warp.similar_items(1, movie_info)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 '584    Aladdin (1992)',
 "2286    Bug's Life, A (1998)",
 '33    Babe (1995)',
 '1245    Groundhog Day (1993)',
 '2252    Pleasantville (1998)',
 '591    Beauty and the Beast (1991)',
 '2502    Matrix, The (1999)',
 '315    Shawshank Redemption, The (1994)']

In [42]:
warp.recommend(4, movie_info)

['847    Godfather, The (1972)',
 '585    Terminator 2: Judgment Day (1991)',
 '1203    Godfather: Part II, The (1974)',
 '2847    Total Recall (1990)',
 '1884    French Connection, The (1971)',
 '2502    Matrix, The (1999)',
 '453    Fugitive, The (1993)',
 '1284    Butch Cassidy and the Sundance Kid (1969)',
 '1182    Aliens (1986)',
 '1568    Hunt for Red October, The (1990)']